<div style="display: flex; justify-content: space-between; align-items: center;">
    <div style="text-align: left; flex: 4;">
        <strong>Author:</strong> Amirhossein Heydari — 
        📧 <a href="mailto:amirhosseinheydari78@gmail.com">amirhosseinheydari78@gmail.com</a> — 
        🐙 <a href="https://github.com/mr-pylin/media-processing-workshop" target="_blank" rel="noopener">github.com/mr-pylin</a>
    </div>
    <div style="display: flex; justify-content: flex-end; flex: 1; gap: 8px; align-items: center; padding: 0;">
        <a href="https://opencv.org/" target="_blank" rel="noopener noreferrer">
            <img src="../assets/images/libraries/opencv/logo/OpenCV_logo_no_text-1.svg"
                 alt="OpenCV Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://pillow.readthedocs.io/" target="_blank" rel="noopener noreferrer">
            <img src="../assets/images/libraries/pillow/logo/pillow-logo-248x250.png"
                 alt="PIL Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://scikit-image.org/" target="_blank" rel="noopener noreferrer">
            <img src="../assets/images/libraries/scikit-image/logo/logo.png"
                 alt="scikit-image Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://scipy.org/" target="_blank" rel="noopener noreferrer">
            <img src="../assets/images/libraries/scipy/logo/logo.svg"
                 alt="SciPy Logo"
                 style="max-height: 48px; width: auto;">
        </a>
    </div>
</div>
<hr>


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Load Images](#toc2_)    
  - [Image Degradation](#toc2_1_)    
- [Multi-Resolution Analysis](#toc3_)    
  - [Motivation](#toc3_1_)    
    - [Limitations of the Fourier Transform](#toc3_1_1_)    
    - [The Wavelet Paradigm](#toc3_1_2_)    
  - [Choosing a Mother Wavelet](#toc3_2_)    
    - [Continuous Wavelets](#toc3_2_1_)    
      - [Morlet](#toc3_2_1_1_)    
      - [Mexican-Hat (Ricker)](#toc3_2_1_2_)    
      - [Shannon](#toc3_2_1_3_)    
    - [Discrete Wavelets](#toc3_2_2_)    
      - [Haar (db1)](#toc3_2_2_1_)    
      - [Daubechies (dbN)](#toc3_2_2_2_)    
      - [Symlets (symN)](#toc3_2_2_3_)    
      - [Coiflets (coifN)](#toc3_2_2_4_)    
      - [Biorthogonal (biorN.Nr) & Reverse Biorthogonal (rbioN.Nr)](#toc3_2_2_5_)    
      - [Meyer Wavelet](#toc3_2_2_6_)    
  - [Continuous Wavelet Transform (CWT)](#toc3_3_)    
    - [Scales, Translations, and Time-Frequency Localization](#toc3_3_1_)    
      - [Scale and Translation](#toc3_3_1_1_)    
      - [Simulation](#toc3_3_1_2_)    
  - [Discrete Wavelet Transform (DWT)](#toc3_4_)    
    - [1-D DWT Concepts](#toc3_4_1_)    
      - [Forward DWT](#toc3_4_1_1_)    
      - [Inverse DWT](#toc3_4_1_2_)    
    - [2-D Wavelet Transform](#toc3_4_2_)    
      - [Separable Filtering & Sub-bands (LL, LH, HL, HH)](#toc3_4_2_1_)    
        - [Single-Level 2-D DWT](#toc3_4_2_1_1_)    
        - [Multi-Level 2-D DWT](#toc3_4_2_1_2_)    
      - [Visualizing the Wavelet Pyramid](#toc3_4_2_2_)    
    - [Stationary Wavelet Transform (SWT)](#toc3_4_3_)    
      - [1-D SWT](#toc3_4_3_1_)    
      - [2-D SWT](#toc3_4_3_2_)    
  - [Applications](#toc3_5_)    
    - [Thresholding and De-noising](#toc3_5_1_)    
      - [Hard Thresholding](#toc3_5_1_1_)    
      - [Soft Thresholding](#toc3_5_1_2_)    
    - [Compression with Wavelets](#toc3_5_2_)    
      - [JPEG-2000 Overview](#toc3_5_2_1_)    
    - [Edge and Texture Analysis](#toc3_5_3_)    
      - [Wavelet-based Edge Detection](#toc3_5_3_1_)    
        - [Using DWT](#toc3_5_3_1_1_)    
        - [Using SWT](#toc3_5_3_1_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)

In [ ]:
import shutil

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pywt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
from numpy.typing import NDArray

In [ ]:
# disable automatic figure display (plt.show() required)  
# this ensures consistency with .py scripts and gives full control over when plots appear
plt.ioff()

In [ ]:
# set NumPy arrays to print wider lines (120 chars)
np.set_printoptions(linewidth=120)

In [ ]:
# create a reproducible random number generator
rng = np.random.default_rng(seed=42)

In [ ]:
# choose animation display method
display_backend = FuncAnimation.to_html5_video if shutil.which("ffmpeg") else FuncAnimation.to_jshtml
display_backend

In [ ]:
# convert to normalized uint8 for display
def to_uint8(arr):
    arr = np.abs(arr)
    return np.clip(arr / arr.max() * 255, 0, 255).astype(np.uint8)

# <a id='toc2_'></a>[Load Images](#toc0_)

In [ ]:
im_1 = cv2.imread("../assets/images/dip_3rd/CH02_Fig0222(b)(cameraman).tif", cv2.IMREAD_GRAYSCALE)
im_2 = cv2.imread("../assets/images/dip_3rd/CH06_Fig0638(a)(lenna_RGB).tif", cv2.IMREAD_COLOR_RGB)

# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), layout="compressed")
axs[0].imshow(im_1, vmin=0, vmax=255, cmap="gray")
axs[0].set_title("CH02_Fig0222(b)(cameraman).tif")
axs[1].imshow(im_2, vmin=0, vmax=255)
axs[1].set_title("CH06_Fig0638(a)(lenna_RGB).tif")
plt.show()

## <a id='toc2_1_'></a>[Image Degradation](#toc0_)

- Check out the [**image-degredation.ipynb**](./utils/image-degredation.ipynb) notebook for more information on the topic of degredation.


In [ ]:
def apply_gaussian_noise(img: NDArray, mean: float = 0, std: float = 25) -> NDArray[np.uint8]:
    gaussian_noise = rng.normal(loc=mean, scale=std, size=img.shape)
    return np.clip(img.astype(np.float64) + gaussian_noise, 0, 255).astype(np.uint8)

In [ ]:
im_1_gaussian_noise = apply_gaussian_noise(im_1, mean=0, std=20)
im_2_gaussian_noise = apply_gaussian_noise(im_2, mean=0, std=50)

# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), layout="compressed")
axs[0].imshow(im_1_gaussian_noise, vmin=0, vmax=255, cmap="gray")
axs[0].set_title("CH02_Fig0222(b)(cameraman).tif")
axs[1].imshow(im_2_gaussian_noise, vmin=0, vmax=255)
axs[1].set_title("CH06_Fig0638(a)(lenna_RGB).tif")
plt.show()

# <a id='toc3_'></a>[Multi-Resolution Analysis](#toc0_)

Multi-resolution analysis (MRA) allows us to examine signals and images at **multiple scales simultaneously**, capturing both **local structures in time/space** and **frequency content**.  

🔍 **Why MRA matters**:

- Classical Fourier analysis only provides **global frequency information**.
- Localized features such as **edges, transients, or textures** are smeared in Fourier analysis.
- MRA provides a **hierarchical decomposition**, enabling **coarse-to-fine analysis**.


## <a id='toc3_1_'></a>[Motivation](#toc0_)

MRA and wavelets arise from the **limitations of classical Fourier analysis** when dealing with **non-stationary or multi-scale signals**.  


### <a id='toc3_1_1_'></a>[Limitations of the Fourier Transform](#toc0_)

📉 **No time localization**:

- The Fourier Transform (FT) decomposes a signal \(x(t)\) into global frequency components:
  
  $$
  X(f) = \int_{-\infty}^{\infty} x(t) \, e^{-j 2 \pi f t} \, dt
  $$

- Each coefficient represents the **overall contribution** of that frequency across the entire signal.
- **Transient events** or sudden changes are spread across all frequencies — you **cannot tell when they occur**.

⚡ **Poor for non-stationary signals**:

- Signals with **time-varying frequencies** are not well represented.
- Sudden jumps, edges, or spikes appear as **artifacts** in the frequency domain.

🌀 **Inefficient for multi-scale analysis**:

- High-frequency short-duration events and low-frequency long-duration trends are **analyzed with the same resolution**, limiting interpretability.

📝 **Docs**:

- `numpy.fft`: [numpy.org/doc/stable/reference/routines.fft.html](http://numpy.org/doc/stable/reference/routines.fft.html)

In [ ]:
t = np.linspace(0, 1, 512)
dt = t[1] - t[0]

# pure sine wave at 10 Hz
pure_signal = np.sin(10 * 2 * np.pi * t)

# sine wave with an abrupt change (discontinuity)
abrupt_signal = pure_signal.copy()
abrupt_signal[256:] += 1  # jump at t = 0.5 s

In [ ]:
# fourier transform
fft_pure = np.fft.fft(pure_signal)
fft_abrupt = np.fft.fft(abrupt_signal)
freqs = np.fft.fftfreq(len(t), d=dt)

In [ ]:
# plot
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16, 4), layout="compressed")

axes[0].plot(t, pure_signal)
axes[0].set(title="Pure Signal", xlabel="Time (s)", ylabel="Amplitude")
axes[1].plot(freqs, np.abs(fft_pure))
axes[1].set(title="Fourier Transform Magnitude", xlabel="Frequency (Hz)", ylabel="Amplitude")
axes[2].plot(t, abrupt_signal)
axes[2].set(title="Discontinuous Signal", xlabel="Time (s)", ylabel="Amplitude")
axes[3].plot(freqs, np.abs(fft_abrupt))
axes[3].set(title="Fourier Transform Magnitude", xlabel="Frequency (Hz)", ylabel="Amplitude")

# add zoomed inset for axes[1] and axes[3]
axins = inset_axes(axes[1], width="40%", height="40%", loc="upper right")
axins.plot(freqs, np.abs(fft_pure), linewidth=1)
axins.set(xlim=[-20, 20], yticks=[])
mark_inset(axes[1], axins, loc1=2, loc2=4, fc="none", ec="0.5")

axins = inset_axes(axes[3], width="40%", height="40%", loc="upper right")
axins.plot(freqs, np.abs(fft_abrupt), linewidth=1)
axins.set(xlim=[-20, 20], yticks=[])
mark_inset(axes[3], axins, loc1=2, loc2=4, fc="none", ec="0.5")

plt.show()

### <a id='toc3_1_2_'></a>[The Wavelet Paradigm](#toc0_)

🌊 **Wavelets provide time-frequency localization**:

- A wavelet is a small, oscillatory function with **zero mean**:

  $$
  \int_{-\infty}^{\infty} \psi(t) \, dt = 0
  $$

- A wavelet must also have **finite energy**:

  $$
  \|\psi(t)\|_2^2 = \int_{-\infty}^{\infty} |\psi(t)|^2 \, dt < \infty
  $$

- These properties ensure that wavelets are **well-localized** and suitable for decomposing signals into components that capture **both time and frequency information**.
- Signals can be decomposed into **scaled and shifted versions** of a single **mother wavelet**:

  $$
  \text{scaled and shifted } \psi(t) \quad \Rightarrow \quad \psi_{a,b}(t) = \frac{1}{\sqrt{|a|}} \, \psi\Big(\frac{t-b}{a}\Big)
  $$

✍️ **Notes**:

- $a$ → scale (controls frequency resolution)
- $b$ → translation (controls time localization)
- $\psi(t)$ → mother wavelet

📝 **Docs**:

- `numpy.trapezoid`: [numpy.org/devdocs/reference/generated/numpy.trapezoid.html](https://numpy.org/devdocs/reference/generated/numpy.trapezoid.html)
- `pywt.ContinuousWavelet`: [pywavelets.readthedocs.io/en/latest/ref/wavelets.html#continuouswavelet-object](https://pywavelets.readthedocs.io/en/latest/ref/wavelets.html#continuouswavelet-object)
- Approximating wavelet functions: [pywavelets.readthedocs.io/en/latest/ref/wavelets.html#approximating-wavelet-functions-continuouswavelet-wavefun](https://pywavelets.readthedocs.io/en/latest/ref/wavelets.html#approximating-wavelet-functions-continuouswavelet-wavefun)

In [ ]:
# define a normalized mother wavelet
mother = pywt.ContinuousWavelet("mexh")
wavelet_data, t = mother.wavefun()

# zero mean
wavelet_mean = np.trapezoid(wavelet_data, t)

# finite energy [normalized]
wavelet_energy = np.trapezoid(wavelet_data**2, t)

# log
print(f"Wavelet mean   : {wavelet_mean:.3e}")
print(f"Wavelet energy : {wavelet_energy:.3f}")

In [ ]:
# plot
plt.plot(t, wavelet_data)
plt.axhline(0, color="black", linestyle="--")
plt.title("Mother Wavelet: Zero Mean & Finite Energy")
plt.xlabel("Time (normalized)")
plt.ylabel("Amplitude")
plt.grid(alpha=0.3)
plt.show()

## <a id='toc3_2_'></a>[Choosing a Mother Wavelet](#toc0_)

- The **mother wavelet** $\psi(t)$ determines how the signal is analyzed.
- Different wavelets emphasize **different signal features**, such as smoothness, oscillations, or edges.

📌 **Key criteria for selection**:

- **Compact support:** The wavelet should be nonzero only in a limited time interval for better localization.
- **Vanishing moments:** Controls the wavelet's ability to capture **polynomial trends**.
- **Symmetry:** Symmetric wavelets reduce **phase distortions**.
- **Frequency response:** Determines whether the wavelet is better for **sharp transitions** or **smooth oscillations**.

⚡ **Trade-off**:

- No single wavelet works best for all signals.
- The choice depends on the **signal characteristics** and the **analysis goal**.

📝 **Docs**:

- Wavelet families: [pywavelets.readthedocs.io/en/latest/ref/wavelets.html#wavelet-families](https://pywavelets.readthedocs.io/en/latest/ref/wavelets.html#wavelet-families)
- Built-in wavelets: [pywavelets.readthedocs.io/en/latest/ref/wavelets.html#built-in-wavelets-wavelist](https://pywavelets.readthedocs.io/en/latest/ref/wavelets.html#built-in-wavelets-wavelist)

In [ ]:
# a list of available built-in wavelet families
pywt.families(short=False)

In [ ]:
# families with short names
pywt.families()

### <a id='toc3_2_1_'></a>[Continuous Wavelets](#toc0_)

<table style="margin:0 auto;">
  <thead>
    <tr>
      <th>Wavelet</th>
      <th>Domain</th>
      <th>Real/Complex</th>
      <th>Orthogonal</th>
      <th>Typical use / remark</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Morlet</td>
      <td>Continuous</td>
      <td>Complex</td>
      <td>❌</td>
      <td>Gaussian-windowed e<sup>i ω t</sup>; classic CWT</td>
    </tr>
    <tr>
      <td>Paul</td>
      <td>Continuous</td>
      <td>Complex</td>
      <td>❌</td>
      <td>Complex Gaussian derivatives; good time resolution</td>
    </tr>
    <tr>
      <td>Mexican-hat / Ricker</td>
      <td>Continuous</td>
      <td>Real</td>
      <td>❌</td>
      <td>2<sup>nd</sup> derivative of Gaussian; real CWT</td>
    </tr>
    <tr>
      <td>Difference-of-Gaussians (DOG)</td>
      <td>Continuous</td>
      <td>Real</td>
      <td>❌</td>
      <td>Real, approx. Mexican-hat</td>
    </tr>
    <tr>
      <td>Shannon</td>
      <td>Continuous</td>
      <td>Complex</td>
      <td>✅</td>
      <td>Sinc in freq.; ideal frequency localisation</td>
    </tr>
    <tr>
      <td>Bump</td>
      <td>Continuous</td>
      <td>Complex</td>
      <td>❌</td>
      <td>Compact support in Fourier domain</td>
    </tr>
    <tr>
      <td>Complex Gaussian</td>
      <td>Continuous</td>
      <td>Complex</td>
      <td>✅</td>
      <td>Higher-order complex Gaussian; orthogonalised</td>
    </tr>
    <tr>
      <td>Gabor wavelet</td>
      <td>Continuous</td>
      <td>Complex</td>
      <td>❌</td>
      <td>Gaussian × complex sinusoid; minimal uncertainty</td>
    </tr>
  </tbody>
</table>


In [ ]:
# a list of available continuous wavelets
pywt.wavelist(kind="continuous")

#### <a id='toc3_2_1_1_'></a>[Morlet](#toc0_)

- The **Morlet wavelet** is a **complex sinusoid modulated by a Gaussian envelope**.
- It is widely used for analyzing **oscillatory signals**.

🌊 **Definition**:

$$
\psi(t) = \pi^{-1/4} \, e^{j \omega_0 t} \, e^{-t^2/2}
$$

- $\omega_0$ → central frequency of the wavelet
- Complex form allows analysis of **both amplitude and phase** of local oscillations

📈 **Characteristics**:

- Excellent **time-frequency localization** for oscillatory patterns
- Captures **fine frequency details** while retaining **temporal information**
- Useful for **EEG, speech, and vibration signals**

✍️ **Notes**:

- The wavelet is **nonzero over a small interval**, ensuring **finite energy**
- Provides a **continuous, smooth representation** in both time and frequency


In [ ]:
# morlet in theory has both real and imaginary parts
t = np.linspace(-5, 5, 500)
omega0 = 5  # central frequency
psi = (np.pi ** -0.25) * np.exp(1j * omega0 * t) * np.exp(-t**2 / 2)

# enable interactive widget mode
%matplotlib widget

# plot
fig = plt.figure(figsize=(15, 4), layout='compressed')

ax1 = fig.add_subplot(1, 3, 1)
ax1.plot(t, psi.real)
ax1.set(title="Real Part", xlabel="Time", ylabel="Amplitude")
ax1.grid(alpha=0.3)

ax2 = fig.add_subplot(1, 3, 2)
ax2.plot(t, psi.imag)
ax2.set(title="Imaginary Part", xlabel="Time", ylabel="Amplitude")
ax2.grid(alpha=0.3)

ax3 = fig.add_subplot(1, 3, 3, projection='3d')
ax3.plot(t, psi.real, psi.imag)
ax3.set(title="3D Complex Representation", xlabel="Time", ylabel="Real", zlabel="Imag")
ax3.grid(alpha=0.3)

plt.show()

In [ ]:
# revert back to default inline mode
%matplotlib inline

In [ ]:
# PyWavelets "morl" is **real-valued only**, not complex
wavelet = pywt.ContinuousWavelet("morl")
psi, x = wavelet.wavefun(level=8)

# plot
plt.figure(figsize=(10, 3))
plt.plot(x, psi)
plt.title("Morlet Wavelet Function")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.grid(alpha=0.3)
plt.show()

#### <a id='toc3_2_1_2_'></a>[Mexican-Hat (Ricker)](#toc0_)

- The **Mexican-Hat wavelet**, also known as the **Ricker wavelet**, is the **second derivative of a Gaussian**.
- It is particularly effective for detecting **peaks and edges** in signals.

🌊 **Definition**:

$$
\psi(t) = \frac{2}{\sqrt{3\sigma} \pi^{1/4}} \left(1 - \frac{t^2}{\sigma^2}\right) e^{-t^2/(2\sigma^2)}
$$

- $\sigma$ → controls the **width** of the wavelet

📈 **Characteristics**:

- **Real-valued** wavelet (no complex part)
- Sensitive to **local maxima and minima** in the signal
- Captures **sharp transitions and edges** effectively
- Useful in **image processing** for **edge detection** and **feature extraction**

✍️ **Notes**:

- The wavelet has **finite energy** and **zero mean**, ensuring proper localization
- Provides a **smooth, bell-shaped response** ideal for multi-resolution analysis


In [ ]:
wavelet = pywt.ContinuousWavelet("mexh")
psi, x = wavelet.wavefun(level=8)

# plot
plt.figure(figsize=(10, 3))
plt.plot(x, psi)
plt.title("Mexican-Hat Wavelet Function")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.grid(alpha=0.3)
plt.show()

#### <a id='toc3_2_1_3_'></a>[Shannon](#toc0_)

- The **Shannon wavelet** is defined in the **frequency domain** and provides **perfect frequency localization**, though it is poorly localized in time.

🌊 **Definition (frequency domain)**:

$$
\Psi(\omega) =
\begin{cases}
1, & \omega_1 \leq |\omega| \leq \omega_2 \\
0, & \text{otherwise}
\end{cases}
$$

- $\omega_1$ and $\omega_2$ define the **passband of the wavelet**

📈 **Characteristics**:

- Ideal for analyzing **band-limited signals**
- Very sharp **frequency resolution**
- Poor **time localization** due to infinite support in time

✍️ **Notes**:

- Useful for **theoretical analysis** and understanding **time-frequency trade-offs**
- Less practical for signals with **rapid temporal changes**, because temporal information is smeared


In [ ]:
wavelet = pywt.ContinuousWavelet("shan0.5-1.0")
psi, x = wavelet.wavefun(level=8)

# plot
plt.figure(figsize=(10, 3))
plt.plot(x, psi.real)
plt.title("Shannon Wavelet Function [Real Part]")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.grid(alpha=0.3)
plt.show()

### <a id='toc3_2_2_'></a>[Discrete Wavelets](#toc0_)

<table style="margin:0 auto;">
  <thead>
    <tr>
      <th>Wavelet</th>
      <th>Domain</th>
      <th>Real/Complex</th>
      <th>Orthogonal</th>
      <th>Linear Phase</th>
      <th>Typical use / remark</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Haar (db1)</td>
      <td>Discrete</td>
      <td>Real</td>
      <td>✅</td>
      <td>✅</td>
      <td>Simplest, oldest DWT filter</td>
    </tr>
    <tr>
      <td>Daubechies db2–db38</td>
      <td>Discrete</td>
      <td>Real</td>
      <td>✅</td>
      <td>❌</td>
      <td>Compact support, max vanishing moments</td>
    </tr>
    <tr>
      <td>Symlets sym2–sym20</td>
      <td>Discrete</td>
      <td>Real</td>
      <td>✅</td>
      <td>Near linear</td>
      <td>Near-symmetric version of db</td>
    </tr>
    <tr>
      <td>Coiflets coif1–coif5</td>
      <td>Discrete</td>
      <td>Real</td>
      <td>✅</td>
      <td>Near linear</td>
      <td>Scaling func. & wavelet have vanishing moments</td>
    </tr>
    <tr>
      <td>Discrete Meyer dmey</td>
      <td>Discrete</td>
      <td>Real</td>
      <td>✅</td>
      <td>❌</td>
      <td>FIR approximation of Meyer continuous wavelet</td>
    </tr>
    <tr>
      <td>Biorthogonal biorX.X</td>
      <td>Discrete</td>
      <td>Real</td>
      <td>❌</td>
      <td>✅</td>
      <td>Different analysis & synthesis; JPEG-2000 default</td>
    </tr>
    <tr>
      <td>Reverse bior rbioX.X</td>
      <td>Discrete</td>
      <td>Real</td>
      <td>❌</td>
      <td>✅</td>
      <td>Filters swapped vs bior</td>
    </tr>
    <tr>
      <td>Complex Gaussian cgau</td>
      <td>Discrete</td>
      <td>Complex</td>
      <td>✅</td>
      <td>❌</td>
      <td>Orthogonalised complex Gaussian</td>
    </tr>
    <tr>
      <td>Complex Morlet cmor</td>
      <td>Discrete</td>
      <td>Complex</td>
      <td>✅</td>
      <td>❌</td>
      <td>Orthogonalised complex Morlet</td>
    </tr>
    <tr>
      <td>Gaussian gau</td>
      <td>Discrete</td>
      <td>Real</td>
      <td>✅</td>
      <td>❌</td>
      <td>Orthogonalised real Gaussian</td>
    </tr>
    <tr>
      <td>Mexican hat mexh</td>
      <td>Discrete</td>
      <td>Real</td>
      <td>✅</td>
      <td>❌</td>
      <td>Orthogonalised Ricker</td>
    </tr>
    <tr>
      <td>Shannon shan</td>
      <td>Discrete</td>
      <td>Complex</td>
      <td>✅</td>
      <td>❌</td>
      <td>Orthogonal sinc-type</td>
    </tr>
    <tr>
      <td>FBSP fbsp</td>
      <td>Discrete</td>
      <td>Complex</td>
      <td>✅</td>
      <td>❌</td>
      <td>Frequency B-spline</td>
    </tr>
  </tbody>
</table>


In [ ]:
# a list of available discrete wavelets
pywt.wavelist(kind="discrete")

#### <a id='toc3_2_2_1_'></a>[Haar (db1)](#toc0_)

- The **Haar wavelet** is the simplest and oldest DWT wavelet.
- It is **piecewise constant** and ideal for detecting **sudden changes** in signals.

🌊 **Definition**:

- Scaling function ($\phi(t)$):

$$
\phi(t) =
\begin{cases}
1, & 0 \le t < 1 \\
0, & \text{otherwise}
\end{cases}
$$

- Wavelet function ($\psi(t)$):

$$
\psi(t) =
\begin{cases}
1, & 0 \le t < \frac{1}{2} \\
-1, & \frac{1}{2} \le t < 1 \\
0, & \text{otherwise}
\end{cases}
$$

📈 **Characteristics**:

- Orthogonal and compactly supported
- Detects **sharp edges and discontinuities**
- Computationally **very efficient**
- Poor frequency resolution due to **blocky nature**

✍️ **Notes**:

- Often used as the **first example in tutorials**
- Forms the foundation for understanding **multi-level DWT decomposition**


In [ ]:
wavelet = pywt.Wavelet("haar")

# get scaling (phi), wavelet (psi), and time axis
phi, psi, x = wavelet.wavefun(level=8)

# plot
fig, axes = plt.subplots(1, 2, figsize=(10, 3), layout="compressed")

axes[0].plot(x, phi, drawstyle="steps-post")
axes[0].set_title("Haar Scaling Function φ(t)")
axes[0].set_xlabel("t")
axes[0].set_ylabel("Amplitude")
axes[0].grid(alpha=0.3)

axes[1].plot(x, psi, drawstyle="steps-post")
axes[1].set_title("Haar Wavelet Function ψ(t)")
axes[1].set_xlabel("t")
axes[1].set_ylabel("Amplitude")
axes[1].grid(alpha=0.3)

plt.show()

#### <a id='toc3_2_2_2_'></a>[Daubechies (dbN)](#toc0_)

- The **Daubechies wavelets** are a family of orthogonal wavelets with **compact support** and **increasing vanishing moments**, denoted as dbN (N = number of vanishing moments).

🌊 **Definition (conceptual)**:

- No simple closed form in time domain (except db1 / Haar)
- Defined through **scaling and wavelet filter coefficients** $(h_k, g_k)$
- Used for multi-resolution decomposition via **filter banks**

📈 **Characteristics**:

- **Orthogonal**: ensures perfect reconstruction
- **Compact support**: good for **time localization**
- **N vanishing moments**: can represent polynomials of degree $N-1$ exactly
- **Asymmetric**: may introduce phase distortion in signal reconstruction

✍️ **Notes**:

- Higher N → smoother wavelets and better frequency selectivity
- Widely used in **image compression**, **denoising**, and **feature extraction**


In [ ]:
wavelet = pywt.Wavelet("db4")

# get scaling (phi), wavelet (psi), and time axis
phi, psi, x = wavelet.wavefun(level=8)

# plot
fig, axes = plt.subplots(1, 2, figsize=(10, 3), layout="compressed")

axes[0].plot(x, phi, drawstyle="steps-post")
axes[0].set_title("Daubechies Scaling Function φ(t)")
axes[0].set_xlabel("t")
axes[0].set_ylabel("Amplitude")
axes[0].grid(alpha=0.3)

axes[1].plot(x, psi, drawstyle="steps-post")
axes[1].set_title("Daubechies Wavelet Function ψ(t)")
axes[1].set_xlabel("t")
axes[1].set_ylabel("Amplitude")
axes[1].grid(alpha=0.3)

plt.show()

#### <a id='toc3_2_2_3_'></a>[Symlets (symN)](#toc0_)

- **Symlets** are a family of nearly **symmetric Daubechies wavelets**, designed to reduce **phase distortion** while preserving **compact support** and **vanishing moments**.

🌊 **Definition (conceptual)**:

- Similar to Daubechies wavelets but modified for **symmetry**
- Defined by **filter coefficients** for decomposition and reconstruction

📈 **Characteristics**:

- Nearly symmetric → **less phase distortion**
- Orthogonal → ensures **perfect reconstruction**
- Compact support → **good time localization**
- N vanishing moments → can represent polynomials of degree $N-1$

✍️ **Notes**:

- Common choice when **signal symmetry is important**
- Slightly smoother than Haar, slightly less asymmetric than Daubechies
- Often used in **image and audio processing** for **feature preservation**


In [ ]:
wavelet = pywt.Wavelet("sym4")

# get scaling (phi), wavelet (psi), and time axis
phi, psi, x = wavelet.wavefun(level=8)

# plot
fig, axes = plt.subplots(1, 2, figsize=(10, 3), layout="compressed")

axes[0].plot(x, phi, drawstyle="steps-post")
axes[0].set_title("Symlets Scaling Function φ(t)")
axes[0].set_xlabel("t")
axes[0].set_ylabel("Amplitude")
axes[0].grid(alpha=0.3)

axes[1].plot(x, psi, drawstyle="steps-post")
axes[1].set_title("Symlets Wavelet Function ψ(t)")
axes[1].set_xlabel("t")
axes[1].set_ylabel("Amplitude")
axes[1].grid(alpha=0.3)

plt.show()

#### <a id='toc3_2_2_4_'></a>[Coiflets (coifN)](#toc0_)

- **Coiflets** are a family of wavelets designed to have **both the wavelet and scaling functions with vanishing moments**, providing **better approximation properties**.

🌊 **Definition (conceptual)**:

- Denoted as coifN, where N determines the **number of vanishing moments**
- Constructed to satisfy:
  - N vanishing moments for the **wavelet function**
  - N vanishing moments for the **scaling function**

📈 **Characteristics**:

- Orthogonal → ensures **perfect reconstruction**
- Nearly symmetric → **reduced phase distortion**
- Compact support → good **time localization**
- Excellent for **polynomial trend analysis**

✍️ **Notes**:

- Often used in **signal and image denoising**
- Preferred when **both approximation and detail coefficients** should have high vanishing moments
- Provides **smooth decomposition** while maintaining reconstruction accuracy


In [ ]:
wavelet = pywt.Wavelet("coif1")

# get scaling (phi), wavelet (psi), and time axis
phi, psi, x = wavelet.wavefun(level=8)

# plot
fig, axes = plt.subplots(1, 2, figsize=(10, 3), layout="compressed")

axes[0].plot(x, phi, drawstyle="steps-post")
axes[0].set_title("Coiflets Scaling Function φ(t)")
axes[0].set_xlabel("t")
axes[0].set_ylabel("Amplitude")
axes[0].grid(alpha=0.3)

axes[1].plot(x, psi, drawstyle="steps-post")
axes[1].set_title("Coiflets Wavelet Function ψ(t)")
axes[1].set_xlabel("t")
axes[1].set_ylabel("Amplitude")
axes[1].grid(alpha=0.3)

plt.show()

#### <a id='toc3_2_2_5_'></a>[Biorthogonal (biorN.Nr) & Reverse Biorthogonal (rbioN.Nr)](#toc0_)

- **Biorthogonal wavelets** provide **linear phase** reconstruction by using **separate decomposition and reconstruction filters**, making them suitable for **image processing applications** like JPEG-2000.

🌊 **Definition (conceptual)**:

- Two sets of filters:
  - **Decomposition filters** $(h, g)$ for analysis
  - **Reconstruction filters** $(\tilde{h}, \tilde{g})$ for synthesis
- Denoted as biorN.Nr or rbioN.Nr, where N, Nr indicate the **order of vanishing moments** for decomposition and reconstruction

📈 **Characteristics**:

- Linear phase → preserves **edges and shapes** in images
- Biorthogonal → allows **perfect reconstruction** even without strict orthogonality
- Compact support → good **time localization**
- Separate decomposition and reconstruction filters → flexibility in **designing smoothness vs. localization**

✍️ **Notes**:

- Widely used in **image compression (JPEG-2000)**
- Choice between bior and rbio depends on **implementation convenience** and **signal symmetry**
- Provides a good balance between **reconstruction accuracy** and **computational efficiency**


In [ ]:
wavelet = pywt.Wavelet("bior2.2")
phi_d, psi_d, phi_r, psi_r, x = wavelet.wavefun(level=8)

fig, axes = plt.subplots(1, 4, figsize=(16, 3), layout="compressed")

axes[0].plot(x, phi_d, drawstyle="steps-post")
axes[0].set_title("φ_d(t) (Scaling - Decomposition)")
axes[0].set_xlabel("t")
axes[0].set_ylabel("Amplitude")
axes[0].grid(alpha=0.3)

axes[1].plot(x, psi_d, drawstyle="steps-post")
axes[1].set_title("ψ_d(t) (Wavelet - Decomposition)")
axes[1].set_xlabel("t")
axes[1].set_ylabel("Amplitude")
axes[1].grid(alpha=0.3)

axes[2].plot(x, phi_r, drawstyle="steps-post")
axes[2].set_title("φ_r(t) (Scaling - Reconstruction)")
axes[2].set_xlabel("t")
axes[2].set_ylabel("Amplitude")
axes[2].grid(alpha=0.3)

axes[3].plot(x, psi_r, drawstyle="steps-post")
axes[3].set_title("ψ_r(t) (Wavelet - Reconstruction)")
axes[3].set_xlabel("t")
axes[3].set_ylabel("Amplitude")
axes[3].grid(alpha=0.3)

plt.show()

#### <a id='toc3_2_2_6_'></a>[Meyer Wavelet](#toc0_)

- The **Meyer wavelet** is an **infinitely differentiable wavelet** defined in the **frequency domain**, providing smooth frequency localization with **compact support in frequency** (but infinite support in time).

🌊 **Definition (frequency domain)**:

$$
\Psi(\omega) =
\begin{cases}
\sin\left(\frac{\pi}{2} \, \nu\left(\frac{3|\omega|}{2\pi} - 1\right)\right) e^{j\omega/2}, & \frac{2\pi}{3} < |\omega| < \frac{4\pi}{3} \\
1, & \frac{4\pi}{3} \le |\omega| \le \frac{8\pi}{3} \\
0, & \text{otherwise}
\end{cases}
$$

- \(\nu(\cdot)\) → smooth transition function

📈 **Characteristics**:

- Infinitely smooth in **frequency domain**
- Compact frequency support → **good frequency selectivity**
- Poor time localization → not ideal for **sharp transients**
- Useful for **theoretical analysis** and **filter design**

✍️ **Notes**:

- Often used in **signal processing research** where smoothness is required
- Less practical for real-time applications due to **infinite time support**


In [ ]:
wavelet = pywt.Wavelet("dmey")

# get scaling (phi), wavelet (psi), and time axis
phi, psi, x = wavelet.wavefun(level=8)

# plot
fig, axes = plt.subplots(1, 2, figsize=(10, 3), layout="compressed")

axes[0].plot(x, phi, drawstyle="steps-post")
axes[0].set_title("Meyer Scaling Function φ(t)")
axes[0].set_xlabel("t")
axes[0].set_ylabel("Amplitude")
axes[0].grid(alpha=0.3)

axes[1].plot(x, psi, drawstyle="steps-post")
axes[1].set_title("Meyer Wavelet Function ψ(t)")
axes[1].set_xlabel("t")
axes[1].set_ylabel("Amplitude")
axes[1].grid(alpha=0.3)

plt.show()

## <a id='toc3_3_'></a>[Continuous Wavelet Transform (CWT)](#toc0_)

The **Continuous Wavelet Transform (CWT)** allows a signal to be analyzed at **all possible scales and translations**, providing a **time-scale representation**.  
Unlike the Fourier transform, it can capture **local frequency changes**.  

🔍 **Definition**:

- For a signal $x(t)$ and a mother wavelet $\psi(t)$, the CWT is defined as:

  $$
  W_x(a,b) = \frac{1}{\sqrt{|a|}} \int_{-\infty}^{\infty} x(t) \, \psi^*\Big(\frac{t-b}{a}\Big) \, dt
  $$

- Here:
  - $a$ → **scale parameter** (controls frequency resolution)
  - $b$ → **translation parameter** (controls time localization)
  - $^*$ → complex conjugate  

⚡ **Interpretation**:

- Small $a$ → **high-frequency components**, captures **fine details**
- Large $a$ → **low-frequency components**, captures **coarse structures**
- Shifting $b$ moves the wavelet along the **time/spatial axis**, detecting features at different locations.

🌀 **Time-Frequency Trade-off**:

- CWT obeys the **uncertainty principle**:

  $$
  \Delta t \cdot \Delta f \gtrsim \text{constant}
  $$

- **Better time resolution** → worse frequency resolution, and vice versa.
- This is why **multi-resolution analysis** is so powerful: it adapts resolution according to scale.

In [ ]:
# morlet mother wavelet
def morlet(t: NDArray[np.float64], f0: int = 1, scale: float = 1, shift: float = 0) -> NDArray[np.float64]:
    ts = (t - shift) / scale
    return np.pi ** (-0.25) * np.exp(-0.5 * ts**2) * np.cos(2 * np.pi * f0 * ts)

In [ ]:
# toy signal
t = np.linspace(-5, 5, 800)
sig = np.sin(2 * np.pi * 0.5 * t) + 0.3 * np.sin(2 * np.pi * 3 * t)

### <a id='toc3_3_1_'></a>[Scales, Translations, and Time-Frequency Localization](#toc0_)

In CWT, the **scale** and **translation** parameters control how the wavelet interacts with the signal:

📏 **Scale $a$**:

- Controls the **width** of the wavelet.
- Small $a$ → **compressed wavelet**, captures **high-frequency, short-duration events**.
- Large $a$ → **stretched wavelet**, captures **low-frequency, long-duration structures**.
- Analogy: zooming in/out in frequency.

↔️ **Translation $b$**:

- Shifts the wavelet along the **time/spatial axis**.
- Allows detection of features at **different time/spatial locations**.
- Provides **localization in time/spatial**.

⚡ **Time-Frequency Resolution**:

- The **time spread** $\Delta t$ and **frequency spread** $\Delta f$ satisfy:

  $$
  \Delta t \cdot \Delta f \gtrsim \frac{1}{4\pi}
  $$

- Small scale
  - Good **time resolution**, poor **frequency resolution**
  - The wavelet is compressed in time, capturing rapid changes, but its frequency content is spread out.

- Large scale
  - Good **frequency resolution**, poor **time resolution**
  - The wavelet is stretched in time, capturing slow variations with precise frequency, but temporal details are blurred.


#### <a id='toc3_3_1_1_'></a>[Scale and Translation](#toc0_)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4), layout="compressed", sharey=True)

for ax in axes:
    ax.plot(t, sig, linewidth=1, alpha=0.5, label="Input Signal")
    ax.set(xlabel="Time", ylim=(-1.5, 2))

(line1,) = axes[0].plot([], [], linewidth=1, label="Shape-only Morlet")
(line2,) = axes[1].plot([], [], linewidth=1, label="Unit-energy Morlet")
(line3,) = axes[2].plot([], [], linewidth=1, label="Unit-energy Morlet")
axes[0].set_ylabel("Amplitude")
axes[0].legend()
axes[1].legend()
axes[2].legend()

plt.close()

In [ ]:
N = 120
scales = np.linspace(0.3, 2.5, N)
shifts = np.linspace(-3, 3, N)


def update(i):
    line1.set_data(t, morlet(t, scale=scales[i], shift=0))
    axes[0].set_title(
        rf"$\psi_{{{scales[i]:.2f},0}}(t)=" rf"\psi\!\left(\frac{{t-0}}{{{scales[i]:.2f}}}\right)$",
        fontsize=16,
        pad=20,
    )
    line2.set_data(t, morlet(t, scale=scales[i], shift=0) / np.sqrt(scales[i]))
    axes[1].set_title(
        rf"$\psi_{{{scales[i]:.2f},0}}(t)="
        rf"\frac{{1}}{{\sqrt{{{scales[i]:.2f}}}}}\,"
        rf"\psi\!\left(\frac{{t-0}}{{{scales[i]:.2f}}}\right)$",
        fontsize=16,
        pad=20,
    )
    line3.set_data(t, morlet(t, scale=1, shift=shifts[i]))
    axes[2].set_title(
        rf"$\psi_{{1, {shifts[i]:.2f}}}(t)="
        rf"\frac{{1}}{{\sqrt{{1}}}}\,"
        rf"\psi\!\left(\frac{{t-{shifts[i]:.2f}}}{{1}}\right)$",
        fontsize=16,
        pad=20,
    )
    return (line1, line2)


anim = FuncAnimation(fig, update, frames=N, interval=50, blit=True)
HTML(display_backend(anim))

#### <a id='toc3_3_1_2_'></a>[Simulation](#toc0_)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4), layout="compressed")

for ax in axes[:2]:
    ax.plot(t, sig, linewidth=1, alpha=0.5, label="Input Signal")
    ax.set(xlabel="Time", ylim=(-1.5, 2))

(l_shift,) = axes[0].plot([], [], linewidth=1, label="Morlet Wavelet")
(l_prod,) = axes[1].plot([], [], linewidth=1, label="point-wise product")
axes[0].legend()
axes[1].legend()

(l_integral,) = axes[2].plot([], [], linewidth=1)
axes[2].set_xlim(-3, 3)
axes[2].set_ylim(-5, 5)
axes[2].set_xlabel("Normalized shift index")
axes[2].set_ylabel("Re ∫ wavelet·signal")
axes[2].set_title("Wavelet Transform Magnitude")

plt.close(fig)

In [ ]:
N = 120
shifts = np.linspace(-3, 3, N)
integrals = np.zeros(N, dtype=np.complex128)


def update(i):
    b = shifts[i]
    wav = morlet(t, scale=1.0, shift=b)
    prod = wav * sig
    integrals[i] = np.sum(prod)

    # update curves
    l_shift.set_data(t, wav.real)
    l_prod.set_data(t, prod.real)
    l_integral.set_data(shifts[: i + 1], integrals[: i + 1].real)

    fig.suptitle(f"a = 1,  b = {b:.2f}")
    return l_shift, l_prod, l_integral


ani = FuncAnimation(fig, update, frames=N, interval=50, blit=True)
HTML(display_backend(ani))

## <a id='toc3_4_'></a>[Discrete Wavelet Transform (DWT)](#toc0_)

- The **Discrete Wavelet Transform (DWT)** provides a **multi-resolution decomposition** of signals using **discrete scales and translations**.
- Unlike CWT, DWT is computationally efficient and widely used in **signal and image compression, denoising, and feature extraction**.

🔍 **Definition**:

- A 1-D signal $x[n]$ can be decomposed using **scaling ($\phi$)** and **wavelet ($\psi$)** functions:

$$
x[n] = \sum_{k} c_{J,k} \, \phi_{J,k}[n] + \sum_{j=1}^{J} \sum_{k} d_{j,k} \, \psi_{j,k}[n]
$$

- Where:
  - $c_{J,k}$ → approximation coefficients at scale $J$
  - $d_{j,k}$ → detail coefficients at scale $j$
  - $\phi_{J,k}[n]$ → scaling functions
  - $\psi_{j,k}[n]$ → wavelet functions

⚡ **Key Features**:

- **Multi-resolution decomposition**: signal split into **coarse (approximation)** and **fine (detail)** components
- **Efficient computation** using **filter banks and downsampling**
- **Perfect reconstruction** is possible if wavelets satisfy **orthogonality or biorthogonality**
- Forms the foundation for **image compression (JPEG-2000)** and **denoising applications**

✍️ **Notes**:

- DWT works on **dyadic scales**: $a = 2^j$, $b = k 2^j$ for integers $j, k$
- Provides a **hierarchical, pyramid-like representation** of the signal

### <a id='toc3_4_1_'></a>[1-D DWT Concepts](#toc0_)

- The **1-D Discrete Wavelet Transform (DWT)** decomposes a discrete signal into **approximation** and **detail** components using a pair of **low-pass** and **high-pass filters**, followed by **downsampling by 2**.

🧩 **Conceptual overview**:

At each level of decomposition:

- The **low-pass filter (scaling filter)** extracts the **slow-varying** or **coarse** information → approximation coefficients $a_j[n]$
- The **high-pass filter (wavelet filter)** captures **rapid changes** or **fine details** → detail coefficients $d_j[n]$
- Both outputs are **downsampled by 2** to reduce redundancy and maintain constant data length

📈 **Hierarchical structure**:

This process can be applied recursively on the approximation coefficients to produce a **multi-level decomposition**:

$$
f[n] \;\xrightarrow{\text{DWT}}\; \{ a_J[n], \; d_J[n], \; d_{J-1}[n], \dots, d_1[n] \}
$$

where:
- $a_J[n]$: coarse approximation at scale $J$
- $d_j[n]$: details at intermediate scales

✍️ **Notes**:

- Provides a **time-frequency representation** with **dyadic resolution** (power-of-two scales)
- Basis functions are **localized** in both time and frequency
- Enables **perfect reconstruction** when filters satisfy orthogonality or biorthogonality

📝 **Docs**:

- Single level dwt: [pywavelets.readthedocs.io/en/latest/ref/dwt-discrete-wavelet-transform.html#single-level-dwt](https://pywavelets.readthedocs.io/en/latest/ref/dwt-discrete-wavelet-transform.html#single-level-dwt)
- Single level idwt: [pywavelets.readthedocs.io/en/latest/ref/idwt-inverse-discrete-wavelet-transform.html#single-level-idwt](https://pywavelets.readthedocs.io/en/latest/ref/idwt-inverse-discrete-wavelet-transform.html#single-level-idwt)
- Multilevel decomposition using wavedec: [pywavelets.readthedocs.io/en/latest/ref/dwt-discrete-wavelet-transform.html#multilevel-decomposition-using-wavedec](https://pywavelets.readthedocs.io/en/latest/ref/dwt-discrete-wavelet-transform.html#multilevel-decomposition-using-wavedec)
- Multilevel reconstruction using waverec: [pywavelets.readthedocs.io/en/latest/ref/idwt-inverse-discrete-wavelet-transform.html#multilevel-reconstruction-using-waverec](https://pywavelets.readthedocs.io/en/latest/ref/idwt-inverse-discrete-wavelet-transform.html#multilevel-reconstruction-using-waverec)


In [ ]:
# 1-D signal example
t = np.linspace(0, 1, 64, endpoint=False)
signal_1d = np.sin(2 * np.pi * 5 * t) + 0.5 * np.sin(2 * np.pi * 20 * t)

#### <a id='toc3_4_1_1_'></a>[Forward DWT](#toc0_)

For a discrete signal $f[n]$, the forward transform is defined as:

$$
a_j[k] = \sum_{n} f[n] \, h[2k - n]
$$
$$
d_j[k] = \sum_{n} f[n] \, g[2k - n]
$$

where:

- $h[n]$: low-pass (scaling) filter  
- $g[n]$: high-pass (wavelet) filter  
- $a_j[k]$: approximation coefficients  
- $d_j[k]$: detail coefficients  


In [ ]:
# forward DWT (decomposition)
wavelet = "db4"
coeffs = pywt.wavedec(signal_1d, wavelet, level=3)
cA3, cD3, cD2, cD1 = coeffs

In [ ]:
# plot
fig, axes = plt.subplots(5, 1, figsize=(12, 8), layout="compressed")

axes[0].plot(signal_1d)
axes[0].set_title("1D Signal")

axes[1].plot(cA3)
axes[1].set_title("Approximation Coefficients (A3)")

axes[2].plot(cD3)
axes[2].set_title("Detail Coefficients (D3)")

axes[3].plot(cD2)
axes[3].set_title("Detail Coefficients (D2)")

axes[4].plot(cD1)
axes[4].set_title("Detail Coefficients (D1)")

plt.show()

#### <a id='toc3_4_1_2_'></a>[Inverse DWT](#toc0_)

Reconstruction is achieved by **upsampling** and **filtering**:

$$
f[n] = \sum_{k} \big( a_j[k] \, \tilde{h}[n - 2k] + d_j[k] \, \tilde{g}[n - 2k] \big)
$$

where $\tilde{h}[n]$ and $\tilde{g}[n]$ are the **reconstruction filters**, corresponding to synthesis filters in the biorthogonal or orthogonal basis.

✍️ **Notes**

- For **orthogonal wavelets**, $\tilde{h}[n] = h[n]$ and $\tilde{g}[n] = g[n]$.


In [ ]:
# cA3, cD3, cD2, cD1 from pywt.wavedec
coeffs = [cA3, cD3, cD2, cD1]

# inverse DWT (reconstruction)
signal_rec = pywt.waverec(coeffs, wavelet)  # use the same wavelet at the forward DWT

# log
error = np.linalg.norm(signal_1d - signal_rec)
print("Reconstruction error:", error)

In [ ]:
# plot
fig, axes = plt.subplots(2, 1, figsize=(12, 4), layout="compressed")

axes[0].plot(signal_1d)
axes[0].set_title("Original 1D Signal")

axes[1].plot(signal_rec)
axes[1].set_title("Reconstructed Signal (Inverse DWT)")

plt.show()

### <a id='toc3_4_2_'></a>[2-D Wavelet Transform](#toc0_)

The **2-D Discrete Wavelet Transform (2-D DWT)** extends the 1-D DWT concept to two-dimensional data such as images.  
It applies **separable filtering** along rows and columns, producing a **multi-resolution representation** of spatial frequencies.

🧩 **Conceptual Overview**

For an image $f(x, y)$:

1. Apply **low-pass** and **high-pass** filters along **rows** → produces intermediate results.
2. Apply the same filters along **columns** to each intermediate result.
3. The output is divided into **four sub-bands** representing different frequency components and orientations.

Mathematically:

$$
f(x, y)
\xrightarrow{\text{row filtering}}
\begin{cases}
L_r(x, y), \\
H_r(x, y)
\end{cases}
\xrightarrow{\text{column filtering}}
\begin{cases}
LL, LH, HL, HH
\end{cases}
$$

📝 **Docs**:

- Single level dwt2: [pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#single-level-dwt2](https://pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#single-level-dwt2)
- Single level idwt2: [pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#single-level-idwt2](https://pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#single-level-idwt2)
- 2D multilevel decomposition using wavedec2: [pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#d-multilevel-decomposition-using-wavedec2](https://pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#d-multilevel-decomposition-using-wavedec2)
- 2D multilevel reconstruction using waverec2: [pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#d-multilevel-reconstruction-using-waverec2](https://pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#d-multilevel-reconstruction-using-waverec2)


#### <a id='toc3_4_2_1_'></a>[Separable Filtering & Sub-bands (LL, LH, HL, HH)](#toc0_)

The 2-D DWT uses **separable filtering**, meaning the **2-D transform** can be implemented as **two successive 1-D transforms** — one along **rows**, then along **columns**.

🧮 **Separable Filtering**

Given 1-D low-pass and high-pass filters $h[n]$ and $g[n]$:

- Apply filters along rows:
  $$
  L_r(x, y) = f(x, y) * h(x)
  $$
  $$
  H_r(x, y) = f(x, y) * g(x)
  $$

- Then along columns:
  $$
  LL = L_r(x, y) * h(y), \quad LH = L_r(x, y) * g(y)
  $$
  $$
  HL = H_r(x, y) * h(y), \quad HH = H_r(x, y) * g(y)
  $$

Each combination produces one of the **four sub-bands**.

🧭 **Sub-band Interpretation**

| Sub-band | Filter Combination | Frequency Content | Visual Meaning |
|-----------|--------------------|------------------|----------------|
| **LL** | Low-Low | Low frequency (smooth) | Approximation |
| **LH** | Low-High | Horizontal edges | Detail |
| **HL** | High-Low | Vertical edges | Detail |
| **HH** | High-High | Diagonal edges | Detail |

✍️ **Notes**

- Each sub-band is **downsampled by 2** along both axes → image size is reduced by 1/4.
- Only the **LL** sub-band is used for the next level of decomposition.
- The separable structure allows efficient implementation using **filter banks**.


##### <a id='toc3_4_2_1_1_'></a>[Single-Level 2-D DWT](#toc0_)


In [ ]:
# perform 1-level 2D DWT [gray-scale image]
wavelet = "db1"
cA, (cH, cV, cD) = pywt.dwt2(im_1, wavelet)

# convert to normalized uint8
cA_u, cH_u, cV_u, cD_u = map(to_uint8, (cA, cH, cV, cD))

# plot
titles = [
    "Original",
    "Approximation (cA_u) (LL)",
    "Horizontal Detail (cH_u) (LH)",
    "Vertical Detail (cV_u) (HL)",
    "Diagonal Detail (cD_u) (HH)",
]
coeffs = [im_1, cA_u, cH_u, cV_u, cD_u]

plt.figure(figsize=(14, 4), layout="compressed")
for i, (title, coeff) in enumerate(zip(titles, coeffs)):
    plt.subplot(1, 5, i + 1)
    plt.imshow(coeff, cmap="gray")
    plt.title(title)

plt.show()

In [ ]:
# perform 1-level 2D DWT [rgb image]
wavelet = "db1"
cA_R, (cH_R, cV_R, cD_R) = pywt.dwt2(im_2[..., 0], wavelet)
cA_G, (cH_G, cV_G, cD_G) = pywt.dwt2(im_2[..., 1], wavelet)
cA_B, (cH_B, cV_B, cD_B) = pywt.dwt2(im_2[..., 2], wavelet)

# merge channels into RGB arrays (H,W,3)
cA = np.stack((cA_R, cA_G, cA_B), axis=-1)
cH = np.stack((cH_R, cH_G, cH_B), axis=-1)
cV = np.stack((cV_R, cV_G, cV_B), axis=-1)
cD = np.stack((cD_R, cD_G, cD_B), axis=-1)

# convert to normalized uint8
cA_u, cH_u, cV_u, cD_u = map(to_uint8, (cA, cH, cV, cD))

# plot
titles = [
    "Original RGB",
    "Approximation (cA_u) (LL)",
    "Horizontal detail (cH_u) (LH)",
    "Vertical detail (cV_u) (HL)",
    "Diagonal detail (cD_u) (HH)",
]
coeffs_u = [im_2, cA_u, cH_u, cV_u, cD_u]

plt.figure(figsize=(14, 4), layout="compressed")
for i, (t, c) in enumerate(zip(titles, coeffs_u)):
    plt.subplot(1, 5, i + 1)
    plt.imshow(c)
    plt.title(t)
plt.show()

##### <a id='toc3_4_2_1_2_'></a>[Multi-Level 2-D DWT](#toc0_)


In [ ]:
# perform 3-level 2D DWT [gray-scale image]
wavelet = "db1"
coeffs = pywt.wavedec2(im_1, wavelet, level=3)

# unpack coefficients
cA3, (cH3, cV3, cD3), (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs


# convert to normalized uint8
cA3_u, cH3_u, cV3_u, cD3_u = map(to_uint8, (cA3, cH3, cV3, cD3))
cH2_u, cV2_u, cD2_u = map(to_uint8, (cH2, cV2, cD2))
cH1_u, cV1_u, cD1_u = map(to_uint8, (cH1, cV1, cD1))

# plot
levels = [[cA3_u, cH3_u, cV3_u, cD3_u], [None, cH2_u, cV2_u, cD2_u], [None, cH1_u, cV1_u, cD1_u]]
titles = [["LL3", "LH3", "HL3", "HH3"], ["", "LH2", "HL2", "HH2"], ["", "LH1", "HL1", "HH1"]]

plt.figure(figsize=(14, 8), layout="compressed")
max_cols = max(len(l) for l in levels)

for row, (level_coeffs, row_titles) in enumerate(zip(levels, titles)):
    for col, (coeff, title) in enumerate(zip(level_coeffs, row_titles)):
        plt.subplot(len(levels), max_cols, row * max_cols + col + 1)
        if coeff is not None:
            plt.imshow(coeff, cmap="gray")
        plt.title(title, fontsize=10)
        plt.axis("off")

plt.show()

#### <a id='toc3_4_2_2_'></a>[Visualizing the Wavelet Pyramid](#toc0_)
 represents a hierarchical decomposition of an image into multiple **resolution levels**, each containing **approximation** and **detail** sub-bands.  

At each level, only the **LL (approximation)** sub-band is further decomposed, forming a **multi-scale representation**.

🧩 **Hierarchical Structure**

After one level of 2-D DWT:

- Level 1 → $\{ LL_1, LH_1, HL_1, HH_1 \}$

Then recursively apply DWT on the **LL** sub-band:

- Level 2 → $\{ LL_2, LH_2, HL_2, HH_2 \}$
- Level 3 → $\{ LL_3, LH_3, HL_3, HH_3 \}$
- …

The full decomposition forms a **pyramid-like structure**, where resolution decreases with depth:

$$
\text{Image} \rightarrow LL_1 \rightarrow LL_2 \rightarrow \dots \rightarrow LL_J
$$

🧭 **Interpretation**

- **Top of the pyramid (LL\_J)** → Coarse global features  
- **Base sub-bands (LH, HL, HH)** → Fine local details  
- Combining all levels reconstructs the **original image** exactly (if filters satisfy perfect reconstruction).

✍️ **Notes**

- Each level provides a **different scale** of spatial-frequency analysis.
- The **wavelet pyramid** is more **computationally efficient** than full spectral methods.
- This representation is heavily used in **image compression**, **denoising**, and **multi-scale feature extraction**.


In [ ]:
wavelet = "db1"
coeffs = pywt.wavedec2(im_1, wavelet, level=3)

In [ ]:
# start with the coarse approximation
pyr = to_uint8(coeffs[0])

# add the three detail layers
for cH, cV, cD in coeffs[1:]:
    h, w = cH.shape
    # pad the current pyramid to even multiple (needed when sizes are odd)
    pyr = np.pad(
        pyr, ((0, h if pyr.shape[0] % 2 else 0), (0, w if pyr.shape[1] % 2 else 0)), mode="constant", constant_values=0
    )

    # stack the 4 quadrants
    top = np.hstack([pyr, to_uint8(cH)])
    bottom = np.hstack([to_uint8(cV), to_uint8(cD)])
    pyr = np.vstack([top, bottom])

In [ ]:
# plot
fig, axes = plt.subplots(1, 2, figsize=(16, 8), layout="compressed")

axes[0].imshow(im_1, cmap="gray")
axes[0].set_title("Original image")

axes[1].imshow(pyr, cmap="gray")
axes[1].set_title("Wavelet pyramid (level-3 db1) [normalized]")

plt.show()

### <a id='toc3_4_3_'></a>[Stationary Wavelet Transform (SWT)](#toc0_)


- The **Stationary Wavelet Transform (SWT)** is a variant of DWT that **does not downsample** at each level, making it **translation-invariant**.

🧮 **Conceptual Idea**  

- At each level, the input signal (or image) is **filtered but not decimated**.
- Filters are **upsampled** at each level instead of downsampling the signal.
- Multi-scale coefficients retain the **same size as the original signal/image**, which is useful for **edge detection, denoising, and feature extraction**.

✍️ **Notes**  

- SWT is also called **“undecimated DWT”** or **“a trous wavelet transform”**.
- More **computationally expensive** than DWT, since it keeps all coefficients at full resolution.
- Ideal for applications where **shift-invariance** is important, e.g., **edge detection or image denoising**.

📝 **Docs**:

- Multilevel 1D swt: [pywavelets.readthedocs.io/en/latest/ref/swt-stationary-wavelet-transform.html#multilevel-1d-swt](https://pywavelets.readthedocs.io/en/latest/ref/swt-stationary-wavelet-transform.html#multilevel-1d-swt)
- Multilevel 1D iswt: [pywavelets.readthedocs.io/en/latest/ref/iswt-inverse-stationary-wavelet-transform.html#multilevel-1d-iswt](https://pywavelets.readthedocs.io/en/latest/ref/iswt-inverse-stationary-wavelet-transform.html#multilevel-1d-iswt)
- Multilevel 2D swt2: [pywavelets.readthedocs.io/en/latest/ref/swt-stationary-wavelet-transform.html#multilevel-2d-swt2](https://pywavelets.readthedocs.io/en/latest/ref/swt-stationary-wavelet-transform.html#multilevel-2d-swt2)
- Multilevel 2D iswt2: [pywavelets.readthedocs.io/en/latest/ref/iswt-inverse-stationary-wavelet-transform.html#multilevel-2d-iswt2](https://pywavelets.readthedocs.io/en/latest/ref/iswt-inverse-stationary-wavelet-transform.html#multilevel-2d-iswt2)


#### <a id='toc3_4_3_1_'></a>[1-D SWT](#toc0_)


In [ ]:
# 1-D signal example
t = np.linspace(0, 1, 64, endpoint=False)
signal_1d = np.sin(2 * np.pi * 5 * t) + 0.5 * np.sin(2 * np.pi * 20 * t)

In [ ]:
# forward SWT (decomposition)
wavelet = "db4"
coeffs = pywt.swt(signal_1d, wavelet, level=3)
(cA1, cD1), (cA2, cD2), (cA3, cD3) = coeffs

In [ ]:
# plot
fig, axes = plt.subplots(5, 1, figsize=(12, 8), layout="compressed")

axes[0].plot(signal_1d)
axes[0].set_title("1D Signal")

axes[1].plot(cA3)
axes[1].set_title("Approximation Coefficients (A3)")

axes[2].plot(cD3)
axes[2].set_title("Detail Coefficients (D3)")

axes[3].plot(cD2)
axes[3].set_title("Detail Coefficients (D2)")

axes[4].plot(cD1)
axes[4].set_title("Detail Coefficients (D1)")

plt.show()

In [ ]:
# reconstruct signal using inverse SWT
signal_rec = pywt.iswt(coeffs, wavelet)

# log reconstruction error
error = np.linalg.norm(signal_1d - signal_rec)
print(f"Reconstruction error: {error}")

In [ ]:
# plot
fig, axes = plt.subplots(2, 1, figsize=(12, 4), layout="compressed")

axes[0].plot(signal_1d)
axes[0].set_title("Original 1D Signal")

axes[1].plot(signal_rec)
axes[1].set_title("Reconstructed Signal (Inverse SWT)")

plt.show()

#### <a id='toc3_4_3_2_'></a>[2-D SWT](#toc0_)


In [ ]:
# perform 3-level 2D SWT [gray-scale image]
wavelet = "db1"
coeffs = pywt.swt2(im_1, wavelet, level=3)

# unpack coefficients
cA3, (cH3, cV3, cD3) = coeffs[0]
cA2, (cH2, cV2, cD2) = coeffs[1]
cA1, (cH1, cV1, cD1) = coeffs[2]

# convert to normalized uint8
cA3_u, cH3_u, cV3_u, cD3_u = map(to_uint8, (cA3, cH3, cV3, cD3))
cA2_u, cH2_u, cV2_u, cD2_u = map(to_uint8, (cA2, cH2, cV2, cD2))
cA1_u, cH1_u, cV1_u, cD1_u = map(to_uint8, (cA1, cH1, cV1, cD1))

# plot
levels = [[cA3_u, cH3_u, cV3_u, cD3_u], [cA2_u, cH2_u, cV2_u, cD2_u], [cA1_u, cH1_u, cV1_u, cD1_u]]
titles = [["LL3", "LH3", "HL3", "HH3"], ["LL2", "LH2", "HL2", "HH2"], ["LL1", "LH1", "HL1", "HH1"]]

plt.figure(figsize=(14, 8), layout="compressed")
max_cols = max(len(l) for l in levels)

for row, (level_coeffs, row_titles) in enumerate(zip(levels, titles)):
    for col, (coeff, title) in enumerate(zip(level_coeffs, row_titles)):
        plt.subplot(len(levels), max_cols, row * max_cols + col + 1)
        if coeff is not None:
            plt.imshow(coeff, cmap="gray")
        plt.title(title, fontsize=10)
        plt.axis("off")

plt.show()

## <a id='toc3_5_'></a>[Applications](#toc0_)


### <a id='toc3_5_1_'></a>[Thresholding and De-noising](#toc0_)

- Wavelet-based **de-noising** leverages the fact that signal energy is concentrated in a few wavelet coefficients, while noise spreads across all coefficients.  
- By **thresholding** small coefficients, we can suppress noise while preserving important signal structures.

🧮 **Conceptual Idea**

1. Compute the **DWT** of the noisy signal/image.
2. Apply a **threshold** to wavelet coefficients:
   - Coefficients below the threshold → likely noise → set to zero
   - Coefficients above the threshold → likely signal → retain or shrink
3. Perform **inverse DWT** to reconstruct the de-noised signal.

Mathematically, for a wavelet coefficient $w$ and threshold $\lambda$:

$$
\tilde{w} =
\begin{cases}
0, & |w| < \lambda \\
f(w), & |w| \ge \lambda
\end{cases}
$$

where $f(w)$ defines the type of thresholding.

✍️ **Notes**

- Thresholding exploits the **sparsity** of wavelet representations.
- Proper threshold selection is crucial: too high → loss of signal details; too low → residual noise.
- Widely used in **audio denoising, image de-noising, and medical signal processing**.

📝 **Docs**:

- Ideal Spatial Adaptation by Wavelet Shrinkage [Paper]: [jstor.org/stable/2337118](https://www.jstor.org/stable/2337118)

#### <a id='toc3_5_1_1_'></a>[Hard Thresholding](#toc0_)

$$
\tilde{w} =
\begin{cases}
0, & |w| < \lambda \\
w, & |w| \ge \lambda
\end{cases}
$$

- Directly sets small coefficients to zero.
- Preserves large coefficients **unchanged**.
- Can introduce **artifacts** due to abrupt changes.


In [ ]:
# forward dwt
wavelet = "db4"
coeffs = pywt.dwt2(im_1_gaussian_noise, wavelet)

# universal threshold (a.k.a. Donoho–Johnstone threshold)
sigma = 20
lam = sigma * np.sqrt(2 * np.log(im_1.size))

# hard threshold
coeffs_thresh = []
for idx, subband in enumerate(coeffs):

    # cA – keep untouched
    if idx == 0:
        coeffs_thresh.append(subband)

    # (cH, cV, cD)
    else:
        coeffs_thresh.append(tuple(pywt.threshold(detail, lam, mode="hard") for detail in subband))

# inverse dwt
denoised = pywt.waverec2(coeffs_thresh, wavelet)
denoised = np.clip(denoised, 0, 255).astype(np.uint8)

In [ ]:
# plot
fig, ax = plt.subplots(1, 3, figsize=(15, 4))
ax[0].imshow(im_1, cmap="gray")
ax[0].set_title("Original")
ax[1].imshow(im_1_gaussian_noise, cmap="gray")
ax[1].set_title(f"Noisy  σ={sigma}")
ax[2].imshow(denoised, cmap="gray")
ax[2].set_title(f"Hard-λ={lam:.1f}")
for a in ax:
    a.axis("off")
plt.show()

#### <a id='toc3_5_1_2_'></a>[Soft Thresholding](#toc0_)

$$
\tilde{w} =
\begin{cases}
0, & |w| < \lambda \\
\text{sign}(w) \cdot (|w| - \lambda), & |w| \ge \lambda
\end{cases}
$$

- Shrinks large coefficients toward zero by \(\lambda\).
- Produces **smoother results**.
- Often preferred for **images and audio**, reducing ringing and discontinuities.


In [ ]:
# forward dwt
wavelet = "db4"
coeffs = pywt.wavedec2(im_1_gaussian_noise, wavelet, level=3)

# universal threshold
# estimate sigma from the HH1 (finest detail) subband if not known
sigma = np.median(np.abs(coeffs[-1][2])) / 0.6745
lam = sigma * np.sqrt(2 * np.log(im_1.size))

# soft threshold
coeffs_soft = [coeffs[0]]  # keep approx untouched
for cH, cV, cD in coeffs[1:]:
    coeffs_soft.append(tuple(pywt.threshold(d, lam, mode="soft") for d in (cH, cV, cD)))

# inverse dwt
denoised = pywt.waverec2(coeffs_soft, "db4")
denoised = np.clip(denoised, 0, 255).astype(np.uint8)

In [ ]:
# plot
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(im_1, cmap="gray")
ax[0].set_title("Original")
ax[1].imshow(im_1_gaussian_noise, cmap="gray")
ax[1].set_title(f"Noisy  σ={sigma}")
ax[2].imshow(denoised, cmap="gray")
ax[2].set_title(f"Soft λ={lam:.1f}")
for a in ax:
    a.axis("off")
plt.tight_layout()
plt.show()

### <a id='toc3_5_2_'></a>[Compression with Wavelets](#toc0_)

- Wavelet-based compression exploits the **sparsity of wavelet coefficients** to represent images efficiently.  
- Most of the image energy is concentrated in a few **large coefficients**, while small coefficients can be discarded or quantized.


#### <a id='toc3_5_2_1_'></a>[JPEG-2000 Overview](#toc0_)

- **JPEG-2000** is a modern image compression standard based on **wavelet transforms** rather than block-based DCT.  

🧩 **Key Features**

- Uses **Discrete Wavelet Transform (DWT)** for multi-resolution decomposition.
- Provides **lossless** and **lossy** compression modes.
- Supports **progressive transmission**, allowing partial image display at lower resolutions.
- Better preserves **edges and textures** compared to traditional JPEG.

🧮 **Compression Pipeline**

1. **Wavelet decomposition**: 2-D DWT produces multiple sub-bands (LL, LH, HL, HH).
2. **Quantization**: Approximation and detail coefficients are quantized.
3. **Encoding**: Efficient coding (e.g., Embedded Zero-Tree Coding) reduces redundancy.
4. **Bitstream generation**: Compressed data is stored or transmitted.

✍️ **Notes**

- JPEG-2000 can achieve **higher compression ratios** with **less visual distortion**.
- Multi-resolution property enables **scalable image representation**.
- Widely used in **medical imaging, remote sensing, and archival storage**.

📝 **Docs**:

- Overview of JPEG 2000: [jpeg.org/jpeg2000/](https://jpeg.org/jpeg2000/)


In [ ]:
# 3-level 2-D DWT (CDF 9/7 ≈ JP2 reversible)
wavelet = "bior4.4"
coeffs = pywt.wavedec2(im_1, wavelet, level=3)

In [ ]:
# unify into one pyramid (like JP2 code-blocks)
pyr, slices = pywt.coeffs_to_array(coeffs)

In [ ]:
# dead-zone uniform quantizer
step = 16  # quantization step (lossy control)
q = np.sign(pyr) * np.floor(np.abs(pyr) / step)  # dead-zone around 0

In [ ]:
# crude embedded coding [encode bit-planes from MSB to LSB]
max_val = int(np.abs(q).max()) + 1
bitplanes = [np.bitwise_and(np.abs(q).astype(np.uint16), 1 << bp) > 0 for bp in range(int(np.log2(max_val)), -1, -1)]

In [ ]:
# progressive decode : use first N bit-planes
N = 4  # keep only 3 MSB planes
recon_q = np.zeros_like(q, dtype=np.float32)
for bp in bitplanes[:N]:
    recon_q = recon_q * 2 + bp.astype(np.float32)
recon_q *= np.sign(q)  # restore sign

In [ ]:
# de-quantize
recon_pyr = recon_q * step

# force exact original shape
recon_pyr = recon_q * step
recon_pyr = recon_pyr[: pyr.shape[0], : pyr.shape[1]]  # guarantee same size

# inverse dwt
recon_coeffs = pywt.array_to_coeffs(recon_pyr, slices, output_format="wavedec2")
recon = pywt.waverec2(recon_coeffs, wavelet)
recon = np.clip(recon, 0, 255).astype(np.uint8)

In [ ]:
# plot
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].imshow(im_1, cmap="gray")
ax[0].set_title("Original")
ax[1].imshow(recon, cmap="gray")
ax[1].set_title(f"JP2-like (step={step}, {N} MSB planes)")
for a in ax:
    a.axis("off")
plt.tight_layout()
plt.show()

### <a id='toc3_5_3_'></a>[Edge and Texture Analysis](#toc0_)

- Wavelets are highly effective for **analyzing edges and textures** because of their **multi-resolution** and **directional sensitivity**.

🧩 **Conceptual Idea**

- Wavelet decomposition separates an image into **approximation** and **detail sub-bands** (LL, LH, HL, HH).
- **Edges** produce large coefficients in **detail sub-bands**, while smooth regions have small coefficients.
- **Textures** correspond to repeating patterns across scales and orientations, captured by wavelet coefficients at multiple levels.

✍️ **Notes**

- Edge detection using wavelets is robust to noise and preserves **directional information**.
- Multi-scale analysis allows capturing **fine and coarse textures** simultaneously.
- Widely used in **image enhancement, pattern recognition, and medical imaging**.


#### <a id='toc3_5_3_1_'></a>[Wavelet-based Edge Detection](#toc0_)

- Wavelet transforms can detect edges by identifying **large wavelet coefficients** in the **detail sub-bands** of an image.

🧮 **Conceptual Idea**

- Apply a **multi-level 2-D DWT** to the image.
- Examine **LH, HL, HH** sub-bands:
  - **LH** → horizontal edges  
  - **HL** → vertical edges  
  - **HH** → diagonal edges
- Threshold or highlight coefficients to locate **edge positions** at different scales.

✍️ **Notes**

- Wavelet edge detection is **multi-scale** → captures both fine and coarse edges.
- More robust to **noise** than gradient-based methods (e.g., Sobel or Prewitt).
- Often used in **image segmentation, feature extraction, and object recognition**.


##### <a id='toc3_5_3_1_1_'></a>[Using DWT](#toc0_)


In [ ]:
wavelet = "db1"
coeffs = pywt.wavedec2(im_1, wavelet, level=3)
num_levels = len(coeffs) - 1  # exclude approximation

In [ ]:
# plot
fig, axes = plt.subplots(1, num_levels, figsize=(5 * num_levels, 5))

for lvl in range(1, num_levels + 1):
    cH, cV, cD = coeffs[-lvl]
    edges = np.abs(cH) + np.abs(cV) + np.abs(cD)
    edges_up = cv2.resize(edges, dsize=(im_1.shape[1], im_1.shape[0]), interpolation=cv2.INTER_LINEAR)
    edges_up = edges_up / edges_up.max()  # normalize
    axes[lvl - 1].imshow(edges_up, cmap="gray")
    axes[lvl - 1].set_title(f"Edges Level {lvl}")

plt.show()

##### <a id='toc3_5_3_1_2_'></a>[Using SWT](#toc0_)


In [ ]:
wavelet = "db1"
coeffs = pywt.swt2(im_1, wavelet, level=3)
num_levels = len(coeffs)  # all levels have full-size arrays

In [ ]:
# plot
fig, axes = plt.subplots(1, num_levels, figsize=(5 * num_levels, 5))

for lvl in range(1, num_levels + 1):
    A, (H, V, D) = coeffs[-lvl]
    edges = np.abs(H) + np.abs(V) + np.abs(D)
    edges = edges / edges.max()
    ax = axes[lvl - 1] if num_levels > 1 else axes
    ax.imshow(edges, cmap="gray")
    ax.set_title(f"Edges Level {lvl}")
    ax.axis("off")

plt.show()